# Advanced topics in Python programming

This notebook explores (slightly) more advanced elements of programming in Python. Among other things we will look into
- functional programming 
- generators/iterators
- documenting code 
- error handling

One aspect not addressed here is writting code following the PEP8 style guide, e.g. indentations, class/function names. There are tools to format your code that way (often given as extension to IDEs), for example [flake8](https://flake8.pycqa.org/en/latest/).

## Bits of functional programming

We have seen how to define a function, i.e. give it a name

```python
def identity(x): return x
```

However, sometimes we might have use for nameless function - enter **Anonymous functions**. For example in sorting

In [1]:
import random

# Let's generate random pairs with the idea of sorting them later
random_tuples = []
for i in range(10):
    random_tuples.append((random.random(), random.random()))
random_tuples

[(0.8504576418472447, 0.1410207543737827),
 (0.20884709230059872, 0.5347896039477744),
 (0.0012045480551106102, 0.9422335785124589),
 (0.8511189825985968, 0.5239069096401694),
 (0.6431452463703352, 0.21928967711592806),
 (0.6293348870210508, 0.008731919878143812),
 (0.8924288307638928, 0.46253520784931434),
 (0.052392869119196606, 0.3750623052155475),
 (0.7003503937816798, 0.35279892474370445),
 (0.11651967883129255, 0.0533703336785708)]

In [2]:
# By default tuples are sorted by considering first the first elements, then comparing the rest, i.e.
sorted(random_tuples)

[(0.0012045480551106102, 0.9422335785124589),
 (0.052392869119196606, 0.3750623052155475),
 (0.11651967883129255, 0.0533703336785708),
 (0.20884709230059872, 0.5347896039477744),
 (0.6293348870210508, 0.008731919878143812),
 (0.6431452463703352, 0.21928967711592806),
 (0.7003503937816798, 0.35279892474370445),
 (0.8504576418472447, 0.1410207543737827),
 (0.8511189825985968, 0.5239069096401694),
 (0.8924288307638928, 0.46253520784931434)]

In [3]:
# Treating them as points we might want to consider their l^2 norm
sorted(random_tuples, key=lambda t: (t[0] ** 2 + t[1] ** 2) ** 0.5)

[(0.11651967883129255, 0.0533703336785708),
 (0.052392869119196606, 0.3750623052155475),
 (0.20884709230059872, 0.5347896039477744),
 (0.6293348870210508, 0.008731919878143812),
 (0.6431452463703352, 0.21928967711592806),
 (0.7003503937816798, 0.35279892474370445),
 (0.8504576418472447, 0.1410207543737827),
 (0.0012045480551106102, 0.9422335785124589),
 (0.8511189825985968, 0.5239069096401694),
 (0.8924288307638928, 0.46253520784931434)]

In [4]:
min(random_tuples, key=lambda t: (t[0] ** 2 + t[1] ** 2) ** 0.5)

(0.11651967883129255, 0.0533703336785708)

Here `lambda` is a key word used for defining anonymous functions. It is followed by arguments. Above the function accepts one argument (referred to as t). The function body follows after `:`. Side note, $\lambda$-calculus and its inventor [Alonzo Church](https://en.wikipedia.org/wiki/Alonzo_Church)

__In capturing variables beware of late binding__

In [5]:
# The idea is that foos[1](x) should return x+1
foos = [lambda x: x + n for n in range(5)]
# But ...
for f in foos:
    print(f(0))

4
4
4
4
4


Definition is evaluated at runtime (then n = 4) and not at definition time

In [6]:
# Solution [referred to as currying]
foos = [lambda x, n=n: x + n for n in range(5)]
for f in foos:
    print(f(0))

0
1
2
3
4


Anonymous functions are often used to build **generator**. Here the idea is that we want to compute on demand and not all the answers at once.

In [7]:
selected = filter(lambda p: p[0] < 0.5, random_tuples)
# Not the answers but ...
selected

In [8]:
# Generator are iterated over
next(selected)

(0.20884709230059872, 0.5347896039477744)

In [9]:
# or consumed entirely
for item in selected:
    print(item)

(0.0012045480551106102, 0.9422335785124589)
(0.052392869119196606, 0.3750623052155475)
(0.11651967883129255, 0.0533703336785708)


In [10]:
# Note that we have now exhausted the iterator so that the following attempt to get the next item fails
next(selected)

StopIteration: 

Iterators can be combined to build processing pipelines

In [11]:
# Keep only the elements in iterable for which the function is true
selected = filter(lambda p: p[0] < 0.5, random_tuples)
# Apply sum function to all the elements in iterable
processed = map(sum, selected)
processed

What is the sum of such elements ? One option is 
```python
sum(list(processed))
```
Also ```sum(processed)``` would work but we want to showcase a nice module from the standard library, namely, `functools`.

In [12]:
# Option 1) to comsume and turn into a list
from functools import reduce

# combine first two items of iterable to make the input
# for next round while the other argument is the next item in iterable
reduce(lambda x, y: x + y, processed)

2.28442000966055

**Food for thought:**
1. Could we use `reduce(sum, processed)` above ?
2. What does `functools.partial` do?
3. Which digits are common to all entries obtained by summing tuples in `random_tuples`?

In [13]:
import operator

def digits(num):
    '''Set of digits of the number'''
    return set(filter(str.isdecimal, str(num)))

# To find the answer we reduce by set intersection
reduce(operator.and_, (map(lambda t: digits(t[0]+t[1]), random_tuples)))

{'0', '3'}

Many useful iterators can be constructed using standard library module ``itertools``. Let's do cartesian coordinates

In [14]:
from itertools import product

x = range(1, 5)
y = range(4, 12)
grid = product(x, y)
# Get them all
print(list(grid))

[(1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11)]


Using `itertools.permutations`, `itertools.combinations` we can save ourselves writing a lot to nested for loop. For example: *Which number 1 <= a < b < c <= 100 for Pythagorean trippets?*  

In [16]:
tripplets = []
for a in range(1, 101):
    for b in range(a, 101):
        for c in range(b, 101):
            # Short circuit
            # arg0 and arg1 evals arg1 only if arg0 is True because
            # arg0 False determined the boolean value of the expression
            # Alternative to if 
            a**2 + b**2 == c**2 and tripplets.append((a, b, c)) 

Contrast with

In [17]:
from itertools import combinations

tripplets2 = []
for (a, b, c) in combinations(range(1, 101), 3):
    a**2 + b**2 == c**2 and tripplets2.append((a, b, c)) 
# Or even shorter with list comprehensions, see later
set(tripplets) == set(tripplets2)

True

Another example of ondemand/lazy computations are **generators**. They are introduced by **yield** keyword.

In [18]:
def even_numbers():
    k = 1
    while True:
        yield 2*k
        k += 1

In [19]:
evens = even_numbers()

We can start consuming the generator. For example take the first items

In [20]:
# NOTE: zip - pairs iterables into tuples, terminating when one of them is exhaused [range(5) determines this above]
# We can run this many times.
for v, _ in zip(evens, range(5)):  # And some more ...
    print(v)

2
4
6
8
10


Here we use itertools's `count` which is counts from argument upwards

In [21]:
from itertools import count  # range()

def odd_numbers():
    for v in count(1):
        yield 2*v-1

In [22]:
for v, _ in zip(odd_numbers(), range(5)):
    print(v)

1
3
5
7
9


In the previous example we were consuming other iterator which was built in `count`. Would the definition of `count` below accomplish the same?

In [25]:
def count(n):
    yield n
    yield from count(n + 1)

numbers = count(-10)
for i in range(10_000):     # How about counting longer (1_000) ?
    print(next(numbers))

-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269

RecursionError: maximum recursion depth exceeded

The issue with our definition is that it is recursive and the calls go to stack 

In [26]:
import sys
sys.getrecursionlimit()
# sys.setrecursionlimit(3_000)

3000

However, increasing the stack size doesn't always cut it. The recursive implementation often yields to exponentially growing execution time (and memory). The notorious example of this is (doubly recursive) Fibonacci numbers.

In [27]:
from itertools import count

def fib(k):
    assert k > 0
    if k == 1:
        return 1
    if k == 2:
        return 1
    return fib(k-1) + fib(k-2)


def fibs():
    '''Generator for all of them'''
    for k in count(1):
        yield fib(k)

Let's look at peformance

In [29]:
from time import perf_counter
from itertools import islice  # Iterator slice  

t0 = perf_counter()
for i, k in enumerate(islice(fibs(), 40)):
    t1 = perf_counter()
    print(f'{i} -> {k} in {(t1 - t0):.5f}s')
    t0 = t1

0 -> 1 in 0.00008s
1 -> 1 in 0.00005s
2 -> 2 in 0.00001s
3 -> 3 in 0.00001s
4 -> 5 in 0.00001s
5 -> 8 in 0.00001s
6 -> 13 in 0.00001s
7 -> 21 in 0.00001s
8 -> 34 in 0.00002s
9 -> 55 in 0.00002s
10 -> 89 in 0.00004s
11 -> 144 in 0.00007s
12 -> 233 in 0.00010s
13 -> 377 in 0.00016s
14 -> 610 in 0.00024s
15 -> 987 in 0.00039s
16 -> 1597 in 0.00065s
17 -> 2584 in 0.00102s
18 -> 4181 in 0.00234s
19 -> 6765 in 0.00166s
20 -> 10946 in 0.00237s
21 -> 17711 in 0.00354s
22 -> 28657 in 0.00675s
23 -> 46368 in 0.01080s
24 -> 75025 in 0.01753s
25 -> 121393 in 0.02921s
26 -> 196418 in 0.04630s
27 -> 317811 in 0.07576s
28 -> 514229 in 0.12999s
29 -> 832040 in 0.19770s
30 -> 1346269 in 0.29997s
31 -> 2178309 in 0.48482s
32 -> 3524578 in 0.86591s
33 -> 5702887 in 1.44713s
34 -> 9227465 in 2.27206s
35 -> 14930352 in 3.65843s
36 -> 24157817 in 5.70565s
37 -> 39088169 in 8.86239s
38 -> 63245986 in 16.77639s


KeyboardInterrupt: 

After a while the time till next number behaves like the Fibonacci sequence itself! We are better off with iterations:

In [30]:
def fibs():
    """Generate Fibonacci numbers"""
    a, b = 0, 1
    while True:
        a, b = a + b, a
        yield a  # Yield keyword makes this function a generator

numbers = fibs()

In [32]:
# Let get first ten
for i, num in zip(range(50), numbers):
    print(i, num)

0 8
1 13
2 21
3 34
4 55
5 89
6 144
7 233
8 377
9 610
10 987
11 1597
12 2584
13 4181
14 6765
15 10946
16 17711
17 28657
18 46368
19 75025
20 121393
21 196418
22 317811
23 514229
24 832040
25 1346269
26 2178309
27 3524578
28 5702887
29 9227465
30 14930352
31 24157817
32 39088169
33 63245986
34 102334155
35 165580141
36 267914296
37 433494437
38 701408733
39 1134903170
40 1836311903
41 2971215073
42 4807526976
43 7778742049
44 12586269025
45 20365011074
46 32951280099
47 53316291173
48 86267571272
49 139583862445


When we care about all results of pipeline it might be better/more explicit/readbable to use **comprehensions**. Here we consider list and dictionary comprehensions. 

In [33]:
# f = open("./data/file.txt")
# stuff
# f.close()

with open("./data/file.txt") as stream:
    # NOTE: with invokes a context manager. We want to manage resources;
    # here open a file and then make sure that it is correctly closed no matter what
    # will happen during manipulation, e.g. some error
    lines = [float(line.strip()) for i, line in enumerate(stream) if i % 2]

    # A Dictionary comprehension, create dict mapping row number to value
    d = {i: float(line.strip()) for i, line in enumerate(stream) if i % 2}


# To be compared with
with open("./data/file.txt") as stream:
    iterator = map(
        lambda p: float(p[1].strip()), filter(lambda p: p[0] % 2, enumerate(stream))
    )
    lines_ = list(iterator)
# Check that they are the same. We will come back to the `assert` statement shortly
assert lines == lines_
(d, bool(lines))

({}, True)

**Food for thought:** 
1. Why is the dictionary empty while we clearly have lines as a non-empty list?
2. What is the performance of building list by for-loop versus list comprehensions? [Consider `%%timeit` magic]

In [34]:
%%timeit


def f(string):
    return sum(map(ord, string))


f("IN3110")

442 ns ± 44.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


While there are *set* comprehensions, one has to be careful with "tuple" comprehensions

In [35]:
{l for l in 'aqowngeåonewvewuw'}

{'a', 'e', 'g', 'n', 'o', 'q', 'u', 'v', 'w', 'å'}

In [36]:
(v for v in islice(fibs(), 20) if v % 2)

<generator object <genexpr> at 0x7f48bc6bdc80>

In [37]:
tuple(v for v in islice(fibs(), 20) if v % 2)

(1, 1, 3, 5, 13, 21, 55, 89, 233, 377, 987, 1597, 4181, 6765)

The **with** keyword is an entry point for [contextmanager](https://docs.python.org/3/library/contextlib.html) Its functionality is useful when there is some setup and tear-down needed before and after computations. Typical example is writing to file.

In [38]:
# Only write in caps
class CapsFile:
    # __enter__ and __exit__ special for context manager protocol
    def __init__(self, file_name, mode):
        self.file_obj = open(file_name, mode)
        write = self.file_obj.write
        # Here we replace the file object's write method
        self.file_obj.write = lambda w, write=write: write(w.upper())
        
    def __enter__(self):
        return self.file_obj
    
    def __exit__(self, type, value, traceback):
        self.file_obj.close()

with CapsFile('tes.t', 'w') as out:
    out.write('Hello weltx')
    # raise ValueError

See about the result

In [39]:
cat tes.t

HELLO WELTX

## Writting cleaner functions

(Personal opinion) A good function 1) does what it is supposed to do, 2) does it quickly, 3) is user/developer friendly. Here we will focus on friendlines

Python uses so called duck-typing but we can express our intensions of the input arguments and function output by type annotations. These can be checked by `mypy` (but are not enfoced)

```python
# Following is code included in factorial.py.
def factorial(n: int) -> int:
    if n == 0:
        return 1
    return n*factorial(n-1)

factorial('works?')
```

We run type analysis by
```bash
(in3110) mirok@evalApply:data|$ mypy factorial.py 
```

Role of arguments should be clarified in a docstring of a function (or class). Type can be part of the docstring. We can also include tests via [doctest](https://docs.python.org/3/library/doctest.html). Documentation in the form of for example HTML pages can be generated by [shinx](https://www.sphinx-doc.org/en/master/usage/quickstart.html). The following illustates a doctring with some nonexhaustive tests. For examples of Google-style docstrings see [here](http://sphinxcontrib-napoleon.readthedocs.io/en/latest/example_google.html)

```python
# Part of factorial_doctest.py
def factorial(n: int) -> int:
    '''Return the factorial of n, an exact integer >= 0.

    Args:
       n (int):  n!

    Returns:
       int.  The factorial value::

    >>> factorial(5)
    120
    >>> factorial(0)
    1

    '''
    if n == 0:
        return 1
    return n*factorial(n-1)

```
To run the doctest we execute
```bash
(in3110) mirok@evalApply:data|$ python factorial_doctest.py -v
```

We will come back to testing when discussing Python package development in the next part.

Enforcing the behaviour via exceptions (and their handling). There are several predifined exception types: eg. ValueError, AssertionError, MethodError. We can also define our own type.

In [ ]:
class MyError(BaseException):
    def __init__(self, msg):
        self.msg = msg

    def __str__(self):
        return f'MyError occured with error message "{self.msg}"'

In [41]:
# This is contrived to illustrate the custom exceptions in action.


def factorial(n: int) -> int:
    """Return the factorial of n, an exact integer >= 0.

    Args:
       n (int):  n!

    Returns:
       int.  The factorial value::

    >>> factorial(5)
    120
    >>> factorial(0)
    1
    >>> factorial(-1)
    Traceback (most recent call last):
        ...
    ValueError: Only non-negative inputs are expected
    """
    # Raise AssertionError if the type is wrong
    assert isinstance(n, int)
    # Raise a different exception for negative integers
    if n < 0:
        raise ValueError("Only non-negative inputs are expected")

    if n == 42:
        raise MyError("This is not meant to be")

    if n == 0:
        return 1
    return n * factorial(n - 1)

Handling the raised exceptions. There are several predifined expection types: eg. ValueError, AssertionError, MethodError. We can also define our own type.

In [43]:
val = -4 #  32

try:
    f = factorial(val)
# We will try with the integer value
except AssertionError:
    from math import ceil

    n = ceil(val)
    print(f"Calling instead with {n}")
    f = factorial(n)

# Let's say that for negative we flip the sign
except ValueError:
    n = -val
    print(f"Calling instead with {n}")
    f = factorial(n)

except MyError as e:
    print("42!")

finally:
    # Sieve through here
    pass

Calling instead with 4


## Modifying function behavior
By now we have written function, we have seen functions that take in functions. What we want to do now is to write functions that return __modified__ functions. In our first example we want to write a function which modifies the input function with timing information.

In [49]:
import time
from functools import wraps


def timeit(foo):
    """Return exacution time"""

    @wraps(foo)
    def wrapper(*args, **kwargs):
        then = time.perf_counter()
        result = foo(*args, **kwargs)
        now = time.perf_counter()
        print(f"{foo.__name__} executed in {now-then} s")

        return result

    return wrapper

Here we use the `@wraps` in order to preserve metadata of `foo` (see below). Let's write the function to be timed.

In [50]:
def one_second():
    time.sleep(1)


print(one_second())

timed = timeit(one_second)
# As a **Food for thought** omit the @wraps decorator above and consider what happens with timed.__name__
timed.__name__

None


'one_second'

In [ ]:
print(timed())

A syntacting sugar for applying timeit is via `@`

In [51]:
@timeit
def one_second():
    time.sleep(1)


print(one_second())

one_second executed in 1.001375096006086 s
None


As another example of decorators consider memoization is a technique for caching the function's return value for given input such that it does not need to be computed again. We can test the idea with the functools.lru_cache decorator.

In [53]:
from functools import lru_cache


def slow_factorial(n):
    """Factorial by recursion"""
    if n == 0:
        return 1
    return n * slow_factorial(n - 1)


@lru_cache
def faster_factorial(n):
    return slow_factorial(n)

Let's see about the speed

In [54]:
%timeit slow_factorial(10)

1.04 µs ± 82.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [55]:
%timeit faster_factorial(10)

64.6 ns ± 9.6 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


As an exercise let's write the cache decorator ourselves. However, unlike in `@timeit` we want to make a decorator which takes in an argument which is the cache size. Note that `@lru_cache` has this behavior too. We base our cache on a dictionary

In [56]:
class Cache(dict):
    def __init__(self, size):
        self.size = size

    def __setitem__(self, key, value):
        # Make room
        if len(self) >= self.size:
            # Grab some key
            key = next(iter(self))
            # and remove the entry
            self.pop(key)
        # Set it via parent (dict class)
        super().__setitem__(key, value)

Recall that decorator with arguments is applied as decorator(arguments)(function). That is decorator(arguments) must return a function

In [57]:
from functools import wraps


def cache(size):
    """Memoize"""

    def decorate(foo):
        memory = Cache(size)

        @wraps(foo)
        def wrapper(*args):
            # Lookup arguments. NOTE: here we only assumed positional arguments
            if args in memory:
                return memory[args]
            # Compute and remember
            result = foo(*args)
            memory[args] = result
            return result

        return wrapper

    return decorate


@cache(10)
def faster_factorial2(n):
    return slow_factorial(n)

In [58]:
faster_factorial2(3)

6

In [ ]:
# c = faster_factorial2.__closure__[1]
# c.cell_contents

Let's now speed up Fibinacci with cache

In [59]:
# Speed up Fibinachi with cache
@cache(1_0000)
def fib(k):
    assert k > 0
    if k == 1:
        return 1
    if k == 2:
        return 1
    return fib(k-1) + fib(k-2)


def fibs():
    '''Generator for all of them'''
    for k in count(1):
        yield fib(k)

In [60]:
from time import perf_counter

t0 = perf_counter()
for i, k in enumerate(islice(fibs(), 100)):
    t1 = perf_counter()
    print(f'{i} -> {k} in {(t1 - t0):.5f}s')
    t0 = t1

0 -> 1 in 0.00010s
1 -> 1 in 0.00005s
2 -> 2 in 0.00001s
3 -> 3 in 0.00001s
4 -> 5 in 0.00001s
5 -> 8 in 0.00001s
6 -> 13 in 0.00001s
7 -> 21 in 0.00001s
8 -> 34 in 0.00001s
9 -> 55 in 0.00001s
10 -> 89 in 0.00001s
11 -> 144 in 0.00001s
12 -> 233 in 0.00001s
13 -> 377 in 0.00001s
14 -> 610 in 0.00001s
15 -> 987 in 0.00001s
16 -> 1597 in 0.00001s
17 -> 2584 in 0.00001s
18 -> 4181 in 0.00001s
19 -> 6765 in 0.00001s
20 -> 10946 in 0.00001s
21 -> 17711 in 0.00001s
22 -> 28657 in 0.00001s
23 -> 46368 in 0.00001s
24 -> 75025 in 0.00001s
25 -> 121393 in 0.00001s
26 -> 196418 in 0.00001s
27 -> 317811 in 0.00002s
28 -> 514229 in 0.00001s
29 -> 832040 in 0.00001s
30 -> 1346269 in 0.00001s
31 -> 2178309 in 0.00001s
32 -> 3524578 in 0.00001s
33 -> 5702887 in 0.00001s
34 -> 9227465 in 0.00001s
35 -> 14930352 in 0.00001s
36 -> 24157817 in 0.00001s
37 -> 39088169 in 0.00001s
38 -> 63245986 in 0.00001s
39 -> 102334155 in 0.00001s
40 -> 165580141 in 0.00001s
41 -> 267914296 in 0.00001s
42 -> 433494437 

Some other useful decorators are `@property` in class definitions.

In [61]:
class UnixName:
    """Max 8 characters"""

    def __init__(self, name):
        self.name = name  # NOTE: here were're calling the setter

    # Get
    @property
    def name(self):
        return self._name

    # Set
    @name.setter
    def name(self, name):
        if len(name) > 8:
            name = name[:8]
        self._name = name


(UnixName("Miro").name, UnixName("MiroslavKuchta").name)

('Miro', 'Miroslav')

Some extras for class: we have seen special methods that hook up e.g. to arithmetic '+' (`__add__`) or interact with **with** (`__enter__`). What if you want to make your class iterable? In the example below we would like to iterate over a word in a special way where each letter is (shifter)[https://en.wikipedia.org/wiki/Caesar_cipher]

In [63]:
# __iter__ method
class Ceasar:
    def __init__(self, word, shift=2):
        assert word.isalpha()
        assert word.lower() == word
        self.word = word
        self.shift = shift
        
    def __iter__(self):
        for ch in self.word:
            yield chr(ord(ch) + self.shift)

In [64]:
for l, c in zip('helloworld', Ceasar('helloworld')):
    print(f'{l} -> {c}')

h -> j
e -> g
l -> n
l -> n
o -> q
w -> y
o -> q
r -> t
l -> n
d -> f


Note that many things can be iterated over - `list` produces its items, `str` the individual characters. Then it makes sense to ask for the first element. We know we `list` and `str` we are good
```python
a = [2, 3, 4]
assert a[0] == 2
```
How would we get first element of a say a set?

In [ ]:
things = set(map(chr, range(35, 40)))
things[1]

In [ ]:
# See Peter Norvig's https://github.com/norvig/pytudes
def first(iterable):
    # By iter we make sure we have something for which `next` can be called
    return next(iter(iterable))

first(things)

See more at Peter Norvig's (pytudes)[https://github.com/norvig/pytudes]

# Some review projects

## The let block
In [Julia language](https://docs.julialang.org/en/v1/manual/variables-and-scoping/) the **let** keyword introduces a new scope, i.e. we can e.g. have local variables named as global ones without interference. We would like to do something similar in python. Of course one option is a function - function body is a new scope.

In [3]:
import importlib

a = 2

def let(np=importlib.import_module('numpy')):
    a = 4
    return np.zeros((a, a))
print(let(), a)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] 2


But this `let` is quite cumbersome. We can do [better](# Inspired by https://stackoverflow.com/questions/61371134/python-let-with-local-scopes-debug-printing-and-temporary-variables
)

```python
from contextlib import contextmanager
from inspect import currentframe, getouterframes

@contextmanager
def let(**bindings):
    # 2 because first frame in `contextmanager` is the decorator      
    frame = getouterframes(currentframe(), 2)[-1][0] 
    locals_ = frame.f_locals
    original = {var: locals_.get(var) for var in bindings}
    locals_.update(bindings)
    yield
    locals_.update(original)
    # Cleanup with keeping in scope the vars that were there originally
    for var in set(bindings).difference(locals_):
        del locals_[var]
```
*FIXME: Ask Min about interplay of this with notebook. Until then, demo outside*

## Fun with annotations
Putting some of the lessons learned today and last time together: the following is heavily inspired by talk by David Beazley, see (YouTube)[https://www.youtube.com/watch?v=js_0wjzuMfc&t=3s] for original. We have seen that type annotations can be used for static type checking. Here we would like to "hook" into the type system with (i) our own types and (ii) at runtime. Think for example a function `set_month` that should take only non-empty strings containing [A-Z][a-z]

```python
def set_month(x : AlphaString):
    return x
```

We begin by getting out custom string type by (multiple) inheritance

In [5]:
class Contract:
    @classmethod
    def check(cls, val):
        pass
    
    
class Typed(Contract):
    type = None
    @classmethod
    def check(cls, val):
        assert isinstance(val, cls.type)
        super().check(val)
        
        
class String(Typed):
    type = str
    
    
class NonEmpty(Contract):
    @classmethod
    def check(cls, val):
        assert len(val) > 0
        super().check(val)
        
        
class AlphaString(String, NonEmpty):
    @classmethod
    def check(cls, val):
        assert val.isalpha()
        super().check(val)

AlphaString.check('mkds6')

AssertionError: 

Now we would like to declare the argument type and enforce it. Of course we could put all in type assertions in the function's body but we have in mind using this pattern a lot and don't want to have these hand-written checks at the beginning of our code
```python
def set_month(x : AlphaString):
    return x
```
For modifying function behaviour we have before used decorators ...

In [7]:
from functools import wraps
import inspect 

def checked(func):
    signature = inspect.signature(func) 
    annotations = func.__annotations__     
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        # Bind arguments(names) to their values
        bound = signature.bind(*args, **kwargs)
        for varname, value in bound.arguments.items():
            if varname in annotations:
                # For each annotated argument check the bound value
                typed = annotations[varname]
                typed.check(value)
        # If everything is okay we can call func
        return func(*args, **kwargs)
    return wrapper

In [9]:
@checked
def set_month(x : AlphaString):
    return x

set_month('janua7ry')

AssertionError: 

## More fun with annotations
In C++ we can have functions with the same name but differing by arguments (types or count) and the right function to be called will be determined by the arguments at compile time. In python, this idea can be accomplished e.g. 
by
```python
def foo(a, b=None):
    if b is None:
        b = 2
    if isinstance(b, float):
        return a*b
    return a+b
```

In [10]:
def foo(a, b=None):
    if b is None:
        b = 2
    if isinstance(b, float):
        return a*b
    return a+b

(foo(2, 3), foo(2, 3.), foo(2))

(5, 6.0, 4)

While this works the `if` statements make the code somewhat convoluted. Since we have the type annotations, wouldn't it be nicer if something like this was possible?
```python
def foo(a: int, b: int):
    return a + b

def foo(a: int, b: float):
    return a*b

def foo(a: int):
    return foo(2, a)
```
We take some inspiration from  Python's founder Guido van Rossum's [recipe](https://www.artima.com/weblogs/viewpost.jsp?thread=101605)

In [11]:
class MultiMethod(object):
    def __init__(self, name):
        self.name = name
        self.typemap = {}
        
    def __call__(self, *args):
        '''Dispatch to method based on type of the arguments'''
        types = tuple(arg.__class__ for arg in args) 
        function = self.typemap.get(types, None)
        if function is None:
            raise TypeError("no match")
        return function(*args)
    
    def register(self, types, function):
        '''Add'''
        if types in self.typemap:
            raise TypeError("duplicate registration")
        self.typemap[types] = function

In [12]:
import inspect

_FUNC_TABLE_ = {}

def multimethod(func):
    signature = inspect.signature(func) 
    annotations = func.__annotations__     
    # Pull out type annotations to define the type signature
    type_info = tuple(annotations[arg] for arg in signature.parameters)
    
    name = func.__name__
    mm = _FUNC_TABLE_.get(name)
    if mm is None:
        mm = _FUNC_TABLE_[name] = MultiMethod(name)
    # Register the implementation for that name and the signature
    mm.register(type_info, func)
    return mm

In [13]:
# In action 
@multimethod
def bar(a: int, b: int):
    return a + b

@multimethod
def bar(a: int, b: float):
    return a*b

@multimethod
def bar(a: int):
    return bar(2, a)

In [14]:
(bar(2, 3), bar(2, 3.), bar(2))

(5, 6.0, 4)

## Back to $\lambda$-calculus 
In (Scheme)(https://en.wikipedia.org/wiki/Scheme_(programming_language)) and other Lisp there is are functions [`cons`, `car` and `cdr`](https://www.gnu.org/software/emacs/manual/html_node/eintr/car-_0026-cdr.html) that (for the purpose of this project) behave such that 
```python
car(cons(a, b)) == a
cdr(cond(a, b)) == b
```
If we were to implement the 3 functions based only on this specification we could reach out to list or tuples, i.e. 
rely on some data structure

In [15]:
cons = lambda a, b: (a, b)

car = lambda p: p[0]
cdr = lambda p: p[1]

In [16]:
a, b = 2, 3

car(cons(a, b)) == a
cdr(cons(a, b)) == b

True

In this spirit of $\lambda$ calculus we can as whether we need data structures? Afterall, everyhing should be possible with just functions

In [17]:
def cons(a, b):
    return lambda f: f(a, b)

def car(thing):
    return thing(lambda p, q: p)

def cdr(thing):
    return thing(lambda p, q: q)

In [18]:

a, b = 2, 3

car(cons(a, b)) == a
cdr(cons(a, b)) == b

True

See [Structured and Interpretetion of Computer Programs](https://mitpress.mit.edu/9780262543231/structure-and-interpretation-of-computer-programs/) for more inspiration.